為了讓自己的 Dockerfile 檔寫得更好，可參考 Docker 官網來練習與參考:<br>
Dockerfile reference: https://docs.docker.com/engine/reference/builder/ <br>
docker 官方鏡像: https://github.com/docker-library/official-images        

# 如何選擇鏡像: FROM 

### 範例01

In [ ]:
<h1>Hello, Docker</h1>

In [ ]:
FROM nginx:1.21.0-alpine
ADD index.html /usr/share/nginx/html/index.html

In [ ]:
docker image build -t mynginx-alpine .
docker image ls

# 執行基礎指令: RUN

### 範例01

In [ ]:
FROM ubuntu:21.04
RUN apt-get update
RUN apt-get install -y wget
RUN wget https://github.com/ipinfo/cli/releases/download/ipinfo-2.0.1/ipinfo_2.0.1_linux_amd64.tar.gz
RUN tar zxf ipinfo_2.0.1_linux_amd64.tar.gz
RUN mv ipinfo_2.0.1_linux_amd64 /usr/bin/ipinfo
RUN rm -rf ipinfo_2.0.1_linux_amd64.tar.gz

In [ ]:
FROM ubuntu:21.04
RUN apt-get update && \
    apt-get install -y wget && \
    wget https://github.com/ipinfo/cli/releases/download/ipinfo-2.0.1/ipinfo_2.0.1_linux_amd64.tar.gz && \
    tar zxf ipinfo_2.0.1_linux_amd64.tar.gz && \
    mv ipinfo_2.0.1_linux_amd64 /usr/bin/ipinfo && \
    rm -rf ipinfo_2.0.1_linux_amd64.tar.gz

In [ ]:
docker image build -f Dockerfile.bad -t badimage . 
docker image ls 
docker image history imageId

# 文件的複製: COPY、ADD

### 範例01

In [ ]:
FROM python:3.9.5-alpine3.13
COPY hello.py /app/hello.py

In [ ]:
print("Hello, Docker!")

In [ ]:
docker image build -f Dockerfile-copy -t hello-copy .
docker image history imageId
docker container run -it hello-copy sh

### 範例02

In [ ]:
Dockerfile-add 檔:

In [ ]:
FROM python:3.9.5-alpine3.13
ADD hello.tar.xz /app/

In [ ]:
docker image build -f Dockerfile-add -t hello-add .
docker image history imageId
docker container run -it hello-add sh

# 資料夾的變換操作: WORKDIR

### 範例01

In [ ]:
From python:3.9.5-alpine3.13
WORKDIR /hello/app
COPY hello.py hello.py

In [ ]:
print("Hello, Docker!")

In [ ]:
docker image build -t hello-workdir .
docker container run -it hello-workdir python hello.py
docker container run -it hello-workdir ls

# 構建參數: ARG 

### 範例01

In [ ]:
FROM ubuntu:21.04
ARG VERSION=2.0.1
RUN apt-get update && \
    apt-get install -y wget && \
    wget https://github.com/ipinfo/cli/releases/download/ipinfo-${VERSION}/ipinfo_${VERSION}_linux_amd64.tar.gz && \
    tar zxf ipinfo_${VERSION}_linux_amd64.tar.gz && \
    mv ipinfo_${VERSION}_linux_amd64 /usr/bin/ipinfo && \
    rm -rf ipinfo_${VERSION}_linux_amd64.tar.gz

In [ ]:
docker image build -f Docker-age -t ipinfo-arg --build-arg VERSION=2.0.0 .

In [ ]:
docker image build -f Docker-age -t ipinfo-arg .
docker container run -it ipinfo-arg sh
$ ipinfo version
$ env 

# 環境變數: ENV

<img src="img/6.png">

### 範例01

In [ ]:
FROM ubuntu:21.04
ENV VERSION=2.0.1
RUN apt-get update && \
    apt-get install -y wget && \
    wget https://github.com/ipinfo/cli/releases/download/ipinfo-${VERSION}/ipinfo_${VERSION}_linux_amd64.tar.gz && \
    tar zxf ipinfo_${VERSION}_linux_amd64.tar.gz && \
    mv ipinfo_${VERSION}_linux_amd64 /usr/bin/ipinfo && \
    rm -rf ipinfo_${VERSION}_linux_amd64.tar.gz

In [ ]:
這樣寫是一樣有好處的，之後要改版本只需要改一個地方。不同之前，這次 VERSION 會被保留在鏡像中。
把 image 建起來之後，先進去 shell 看看 ipinfo 的版本是不是跟我們想的一樣。
再輸入 Linux 指令 env 看看環境變量有哪些，在這個範例下會有環境變數 VERSION=2.0.1 。
最後可以試著練習在建立容器的時候用長選項 --env 來改一下 容器內的環境變數。

In [ ]:
docker image build -f Dockerfile-env -t ipinfo-env .
docker container run -it ipinfo-env sh
$ ipinfo version
$ env

# 容器啟動命令: CMD

### 範例01

In [ ]:
FROM ubuntu:21.04
ENV VERSION=2.0.1
RUN apt-get update && \
    apt-get install -y wget && \
    wget https://github.com/ipinfo/cli/releases/download/ipinfo-${VERSION}/ipinfo_${VERSION}_linux_amd64.tar.gz && \
    tar zxf ipinfo_${VERSION}_linux_amd64.tar.gz && \
    mv ipinfo_${VERSION}_linux_amd64 /usr/bin/ipinfo && \
    rm -rf ipinfo_${VERSION}_linux_amd64.tar.gz
CMD ["ipinfo"]

In [ ]:
FROM ubuntu:21.04
ENV VERSION=2.0.1
RUN apt-get update && \
    apt-get install -y wget && \
    wget https://github.com/ipinfo/cli/releases/download/ipinfo-${VERSION}/ipinfo_${VERSION}_linux_amd64.tar.gz && \
    tar zxf ipinfo_${VERSION}_linux_amd64.tar.gz && \
    mv ipinfo_${VERSION}_linux_amd64 /usr/bin/ipinfo && \
    rm -rf ipinfo_${VERSION}_linux_amd64.tar.gz
CMD []

In [ ]:
docker image build -f Dockerfile-cmd -t ipinfo-cmd .
docker container run -it ipinfo-cmd
docker container run -it ipinfo-cmd sh
docker container run -it ipinfo-cmd ipinfo 8.8.8.8
docker system prune -f

# 容器啟動命令: ENTRYPOINT

### 範例01

In [ ]:
FROM ubuntu:21.04
CMD ["echo", "hello docker"]

In [ ]:
FROM ubuntu:21.04
RUN apt-get update && \
    apt-get install -y curl
ENTRYPOINT ["curl", "-s", "www.google.com"]

In [ ]:
FROM ubuntu:21.04
RUN apt-get update && \
    apt-get install -y curl
ENTRYPOINT ["curl", "-s", "www.google.com"]
CMD ["-i"]

### 範例02 - Shell 格式和 Exec 格式

# 容器健康檢查: HEALTHCHECK

### 範例01

In [ ]:
from flask import Flask
from redis import Redis
import os
import socket



app = Flask(__name__)
redis = Redis(host=os.environ.get('REDIS_HOST', '127.0.0.1'), port=6379,
              password=os.environ.get("REDIS_PASS"))

@app.route('/')
def hello():
    redis.incr('hits')
    return f"Hello Container World! I have been seen {redis.get('hits').decode('utf-8')} times and my hostname is {socket.gethostname()}.\n"

In [ ]:
FROM python:3.9.5-slim


RUN pip install flask redis && \
    apt-get update && \
    apt-get install -y curl && \
    groupadd -r flask && useradd -r -g flask flask && \
    mkdir /src && \
    chown -R flask:flask /src
USER flask
COPY app.py /src/app.py
WORKDIR /src
ENV FLASK=app.py REDIS_HOST=redis FLASK_RUN_HOST=0.0.0.0
EXPOSE 5000
HEALTHCHECK --interval=30s --timeout=30s \
    CMD curl -f http://localhost:5000 || exit 1
CMD ["flask", "run"]

In [ ]:
docker image build -t flask-demo .
docker network create -d bridge mybridge

In [ ]:
docker container run -d --network mybridge -p 5000:5000 --env REDIS_PASS=abc123 flask-demo 

In [ ]:
docker container ls

In [ ]:
docker container inspect containerID 

In [ ]:
docker container run -d --network mybridge --name redis redis:latest --requirepass abc123

In [ ]:
docker container ls

# 綜合演練: 構建一個 Python Flask 鏡像

In [ ]:
from flask import Flask


app = Flask(__name__)


@app.route("/")
def index():
    return "Hello, Docter!"

In [ ]:
FROM python:3.9.13-slim

In [ ]:
RUN pip install flask 

In [ ]:
COPY main.py /src/project/main.py

In [ ]:
ENV FLASK_APP=main.py
    FLASK_RUN_HOST=0.0.0.0
    FLASK_RUN_PORT=5000
    FLASK_ENV=develpment
    FLASK_DEBUG=1

In [ ]:
EXPOSE 5000

In [ ]:
WORKDIR /src/project

最終運行我們的 flask 程式。

In [ ]:
CMD ["flask", "run"]

---

In [ ]:
FROM python:3.9.13-slim
RUN pip install flask 
COPY main.py /src/project/main.py
ENV FLASK_APP=main.py
ENV FLASK_RUN_HOST=0.0.0.0
ENV FLASK_RUN_PORT=5000
ENV FLASK_ENV=develpment
ENV FLASK_DEBUG=1
EXPOSE 5000
WORKDIR /src/project
CMD ["flask", "run"]

In [ ]:
docker image build -t flask-demo .
docker container run -p 50:5000 flask-demo

# Dockerfile 的最佳實踐: 使用鏡像緩存

### 範例01

In [ ]:
FROM python:3.9.13-slim
RUN pip install flask 
COPY main.py /src/project/main.py
ENV FLASK_APP=main.py
ENV FLASK_RUN_HOST=0.0.0.0
ENV FLASK_RUN_PORT=5000
ENV FLASK_ENV=develpment
ENV FLASK_DEBUG=1
EXPOSE 5000
WORKDIR /src/project
CMD ["flask", "run"]

In [ ]:
FROM python:3.9.13-slim
RUN pip install flask 
ENV FLASK_APP=main.py
ENV FLASK_RUN_HOST=0.0.0.0
ENV FLASK_RUN_PORT=5000
ENV FLASK_ENV=develpment
ENV FLASK_DEBUG=1
WORKDIR /src/project
COPY main.py main.py
EXPOSE 5000
CMD ["flask", "run"]

# Dockerfile 的最佳實踐: 使用 .dockerignore

In [ ]:
docker image build -t demo:v1 .

### 範例01

In [ ]:
FROM python:3.9.13-slim
RUN pip install flask 
ENV FLASK_APP=main.py
ENV FLASK_RUN_HOST=0.0.0.0
ENV FLASK_RUN_PORT=5000
ENV FLASK_ENV=develpment
ENV FLASK_DEBUG=1
WORKDIR /src/project
COPY main.py main.py
EXPOSE 5000
CMD ["flask", "run"]

In [ ]:
venv
__pycache__

# Dockerfile 的最佳實踐: 鏡像的多階段構建

### 範例01

In [ ]:
#include <stdio.h>

void main(int argc, char *argv[])
{
    printf("hello %s\n", argv[argc - 1]);
}

In [ ]:
FROM gcc:9.4 as builder
COPY hello.c /src/hello.c
WORKDIR /src
RUN gcc --static -o hello hello.c 

FROM alpine:3.13.5
COPY --from=builder /src/hello /src/hello 
ENTRYPOINT ["/src/hello"]
CMD []

In [ ]:
docker image build -t gcc-alpine .
docker container run gcc-alpine world

# Dockerfile 的最佳實踐: 使用非 root 用戶

### 範例01

In [ ]:
FROM python:3.9.5-slim
RUN pip install flask && \
    groupadd -r flask && useradd -r -g flask flask && \
    mkdir /src &&\
    chown -R flask:flask /src

USER flask
COPY app.py /src/app.py
WORKDIR /src
ENV FLASK_APP=main.py
ENV FLASK_RUN_HOST=0.0.0.0
EXPOSE 5000
CMD ["flask", "run"]

In [ ]:
docker image build -t noroot .
docker container run -it noroot sh
$whoami